# Introduction to DataFrames
**[Bogumił Kamiński](http://bogumilkaminski.pl/about/), May 6, 2020**

In [1]:
using DataFrames

## Split-apply-combine

### Grouping a data frame

In [2]:
x = DataFrame(id=[1,2,3,4,1,2,3,4], id2=[1,2,1,2,1,2,1,2], v=rand(8))

,id,id2,v
,Int64,Int64,Float64
1,1,1,0.652452
2,2,2,0.0176934
3,3,1,0.626628
4,4,2,0.106421
5,1,1,0.991339
6,2,2,0.857695
7,3,1,0.268431
8,4,2,0.363826


In [3]:
groupby(x, :id)

,id,id2,v
,Int64,Int64,Float64
1,1,1,0.652452
2,1,1,0.991339
,id,id2,v
,Int64,Int64,Float64
1,4,2,0.106421
2,4,2,0.363826


In [4]:
groupby(x, [])

,id,id2,v
,Int64,Int64,Float64
1,1,1,0.652452
2,2,2,0.0176934
3,3,1,0.626628
4,4,2,0.106421
5,1,1,0.991339
6,2,2,0.857695
7,3,1,0.268431
8,4,2,0.363826


In [5]:
gx2 = groupby(x, [:id, :id2])

,id,id2,v
,Int64,Int64,Float64
1,1,1,0.652452
2,1,1,0.991339
,id,id2,v
,Int64,Int64,Float64
1,4,2,0.106421
2,4,2,0.363826


In [6]:
parent(gx2) # get the parent DataFrame 

,id,id2,v
,Int64,Int64,Float64
1,1,1,0.652452
2,2,2,0.0176934
3,3,1,0.626628
4,4,2,0.106421
5,1,1,0.991339
6,2,2,0.857695
7,3,1,0.268431
8,4,2,0.363826


In [7]:
vcat(gx2...) # back to the DataFrame, but in a different order of rows than the original

,id,id2,v
,Int64,Int64,Float64
1,1,1,0.652452
2,1,1,0.991339
3,2,2,0.0176934
4,2,2,0.857695
5,3,1,0.626628
6,3,1,0.268431
7,4,2,0.106421
8,4,2,0.363826


In [8]:
DataFrame(gx2) # the same

,id,id2,v
,Int64,Int64,Float64
1,1,1,0.652452
2,1,1,0.991339
3,2,2,0.0176934
4,2,2,0.857695
5,3,1,0.626628
6,3,1,0.268431
7,4,2,0.106421
8,4,2,0.363826


In [9]:
DataFrame(gx2, keepkeys=false) # drop grouping columns when creating a data frame

,v
,Float64
1,0.652452
2,0.991339
3,0.0176934
4,0.857695
5,0.626628
6,0.268431
7,0.106421
8,0.363826


In [10]:
groupcols(gx2) # vector of names of grouping variables

2-element Array{Symbol,1}:
 :id
 :id2

In [11]:
valuecols(gx2) # and non-grouping variables

1-element Array{Symbol,1}:
 :v

In [12]:
groupindices(gx2) # group indices in parent(gx2)

8-element Array{Union{Missing, Int64},1}:
 1
 2
 3
 4
 1
 2
 3
 4

In [13]:
kgx2 = keys(gx2)

4-element DataFrames.GroupKeys{GroupedDataFrame{DataFrame}}:
 GroupKey: (id = 1, id2 = 1)
 GroupKey: (id = 2, id2 = 2)
 GroupKey: (id = 3, id2 = 1)
 GroupKey: (id = 4, id2 = 2)

You can index into a `GroupedDataFrame` like to a vector or to a dictionary.
The second form acceps `GroupKey`, `NamedTuple` or a `Tuple`

In [14]:
gx2

,id,id2,v
,Int64,Int64,Float64
1,1,1,0.652452
2,1,1,0.991339
,id,id2,v
,Int64,Int64,Float64
1,4,2,0.106421
2,4,2,0.363826


In [15]:
k = keys(gx2)[1]

GroupKey: (id = 1, id2 = 1)

In [16]:
ntk = NamedTuple(k)

(id = 1, id2 = 1)

In [17]:
tk = Tuple(k)

(1, 1)

the operations below produce the same result and are fast

In [18]:
gx2[1]

,id,id2,v
,Int64,Int64,Float64
1,1,1,0.652452
2,1,1,0.991339


In [19]:
gx2[k]

,id,id2,v
,Int64,Int64,Float64
1,1,1,0.652452
2,1,1,0.991339


In [20]:
gx2[ntk]

,id,id2,v
,Int64,Int64,Float64
1,1,1,0.652452
2,1,1,0.991339


In [21]:
gx2[tk]

,id,id2,v
,Int64,Int64,Float64
1,1,1,0.652452
2,1,1,0.991339


handling missing values

In [22]:
x = DataFrame(id = [missing, 5, 1, 3, missing], x = 1:5)

,id,x
,Int64?,Int64
1,missing,1
2,5,2
3,1,3
4,3,4
5,missing,5


In [23]:
groupby(x, :id) # by default groups include mising values and are not sorted

,id,x
,Int64?,Int64
1,missing,1
2,missing,5
,id,x
,Int64?,Int64
1,3,4


In [24]:
groupby(x, :id, sort=true, skipmissing=true) # but we can change it

,id,x
,Int64?,Int64
1,1,3
,id,x
,Int64?,Int64
1,5,2


### Performing transformations by group using `combine`, `select`, `select!`, `transform`, and `transform!`

In [25]:
using Statistics
using Pipe

In [26]:
ENV["LINES"] = 15 # reduce the number of rows in the output

15

In [27]:
x = DataFrame(id=rand('a':'d', 100), v=rand(100))

,id,v
,Char,Float64
1,'a',0.532229
2,'c',0.234753
3,'d',0.481919
4,'a',0.332086
5,'c',0.170635
6,'b',0.67154
7,'a',0.316082
8,'d',0.225923
9,'c',0.142879


In [28]:
# apply a function to each group of a data frame
# combine keeps as many rows as are returned from the function
@pipe x |> groupby(_, :id) |> combine(_, :v=>mean)

,id,v_mean
,Char,Float64
1,'a',0.55903
2,'c',0.448484
3,'d',0.562074
4,'b',0.584039


In [29]:
x.id2 = axes(x, 1)

Base.OneTo(100)

In [30]:
# select and transform keep as many rows as are in the source data frame and in correct order
# additionally transform keeps all columns from the source
@pipe x |> groupby(_, :id) |> transform(_, :v=>mean)

,id,v,id2,v_mean
,Char,Float64,Int64,Float64
1,'a',0.532229,1,0.55903
2,'c',0.234753,2,0.448484
3,'d',0.481919,3,0.562074
4,'a',0.332086,4,0.55903
5,'c',0.170635,5,0.448484
6,'b',0.67154,6,0.584039
7,'a',0.316082,7,0.55903
8,'d',0.225923,8,0.562074
9,'c',0.142879,9,0.448484


In [31]:
# note that combine reorders rows by group of GroupedDataFrame
@pipe x |> groupby(_, :id) |> combine(_, :id2, :v=>mean)

,id,id2,v_mean
,Char,Int64,Float64
1,'a',1,0.55903
2,'a',4,0.55903
3,'a',7,0.55903
4,'a',12,0.55903
5,'a',30,0.55903
6,'a',32,0.55903
7,'a',40,0.55903
8,'a',47,0.55903
9,'a',49,0.55903


In [32]:
# we give a custom name for the result column
@pipe x |> groupby(_, :id) |> combine(_, :v=>mean=>:res)

,id,res
,Char,Float64
1,'a',0.55903
2,'c',0.448484
3,'d',0.562074
4,'b',0.584039


In [33]:
# you can have multiple operations
@pipe x |> groupby(_, :id) |> combine(_, :v=>mean=>:res1, :v=>sum=>:res2, nrow=>:n)

,id,res1,res2,n
,Char,Float64,Float64,Int64
1,'a',0.55903,11.7396,21
2,'c',0.448484,12.5576,28
3,'d',0.562074,16.8622,30
4,'b',0.584039,12.2648,21


Additional notes:
* `select!` and `transform!` perform operations in-place
* The general syntax for transformation is `source_columns => function => target_column`
* if you pass multiple columns to a function they are treated as positional arguments
* `ByRow` and `AsTable` work exactly like discussed for operations on data frames in 05_columns.ipynb
* you can automatically groupby again the result of `combine`, `select` etc. by passing `ungroup=false` keyword argument to them
* similarly `keepkeys` keyword argument allows you to drop grouping columns from the resulting data frame

As a special case `combine` supports passing a function (or `source_columns => function`) as its first argument. In this case the return value can be a table (not necessarily a single column). In particular it allows for an easy dropping of groups if you return an empty table from the function.

If you pass a function you can use a `do` block syntax. In case of passing a function it gets a `SubDataFrame` as its argument.

Here is an example:

In [34]:
combine(groupby(x, :id)) do sdf
    n = nrow(sdf)
    n < 25 ? DataFrame() : DataFrame(n=n) # drop groups with low number of rows
end

,id,n
,Char,Int64
1,'c',28
2,'d',30


### Aggregation of a data frame using `mapcols`

In [35]:
x = DataFrame(rand(10, 10))

,x1,x2,x3,x4,x5,x6,x7,x8
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.699537,0.690756,0.0202461,0.0441678,0.541186,0.356608,0.946029,0.562003
2,0.859556,0.458333,0.256959,0.765945,0.880905,0.978064,0.740611,0.248021
3,0.444702,0.987374,0.609566,0.279643,0.980609,0.817787,0.480741,0.279605
4,0.757521,0.502225,0.502225,0.38666,0.355246,0.771853,0.413771,0.671273
5,0.20075,0.947523,0.00398774,0.430645,0.599754,0.235945,0.145514,0.725926
6,0.127764,0.266831,0.906741,0.267865,0.0108293,0.637281,0.66997,0.655019
7,0.725646,0.782385,0.594094,0.33725,0.136704,0.352221,0.833425,0.128562
8,0.0748551,0.273572,0.238402,0.77553,0.108553,0.785024,0.316809,0.173973
9,0.578663,0.19875,0.191257,0.243014,0.39303,0.481783,0.37758,0.300757


In [36]:
mapcols(mean, x)

,x1,x2,x3,x4,x5,x6,x7,x8,x9
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.455359,0.553963,0.360706,0.373457,0.464588,0.618623,0.513043,0.37859,0.34916


### Mapping rows and columns using `eachcol` and `eachrow`

In [37]:
map(mean, eachcol(x)) # map a function over each column and return a vector

10-element Array{Float64,1}:
 0.4553587210118186
 0.5539632875669035
 0.3607058090138871
 0.3734565379729605
 0.46458783013334576
 0.6186227076365912
 0.5130433961348055
 0.37859019147738027
 0.34915994633142927
 0.4617615335181361

In [38]:
# an iteration returns a Pair with column name and values
foreach(c -> println(c[1], ": ", mean(c[2])), pairs(eachcol(x)))

x1: 0.4553587210118186
x2: 0.5539632875669035
x3: 0.3607058090138871
x4: 0.3734565379729605
x5: 0.46458783013334576
x6: 0.6186227076365912
x7: 0.5130433961348055
x8: 0.37859019147738027
x9: 0.34915994633142927
x10: 0.4617615335181361


In [39]:
# now the returned value is DataFrameRow which works as a NamedTuple but is a view to a parent DataFrame
map(r -> r.x1/r.x2, eachrow(x))

10-element Array{Float64,1}:
 1.0127120539536034
 1.8753949320995449
 0.4503889233746775
 1.5083303604397296
 0.21186830896522074
 0.4788209282465344
 0.9274800681070121
 0.2736216348742381
 2.9115190621769718
 0.19586662967445118

In [40]:
# it prints like a data frame, only the caption is different so that you know the type of the object
er = eachrow(x)

,x1,x2,x3,x4,x5,x6,x7,x8
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.699537,0.690756,0.0202461,0.0441678,0.541186,0.356608,0.946029,0.562003
2,0.859556,0.458333,0.256959,0.765945,0.880905,0.978064,0.740611,0.248021
3,0.444702,0.987374,0.609566,0.279643,0.980609,0.817787,0.480741,0.279605
4,0.757521,0.502225,0.502225,0.38666,0.355246,0.771853,0.413771,0.671273
5,0.20075,0.947523,0.00398774,0.430645,0.599754,0.235945,0.145514,0.725926
6,0.127764,0.266831,0.906741,0.267865,0.0108293,0.637281,0.66997,0.655019
7,0.725646,0.782385,0.594094,0.33725,0.136704,0.352221,0.833425,0.128562
8,0.0748551,0.273572,0.238402,0.77553,0.108553,0.785024,0.316809,0.173973
9,0.578663,0.19875,0.191257,0.243014,0.39303,0.481783,0.37758,0.300757


In [41]:
er.x1 # you can access columns of a parent data frame directly

10-element Array{Float64,1}:
 0.6995369083921843
 0.859555811172676
 0.44470235873549924
 0.7575213122413824
 0.20075005400832024
 0.12776437720077927
 0.7256461302224049
 0.07485514104662538
 0.5786633548942388
 0.08459176220407594

In [42]:
# it prints like a data frame, only the caption is different so that you know the type of the object
ec = eachcol(x)

,x1,x2,x3,x4,x5,x6,x7,x8
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.699537,0.690756,0.0202461,0.0441678,0.541186,0.356608,0.946029,0.562003
2,0.859556,0.458333,0.256959,0.765945,0.880905,0.978064,0.740611,0.248021
3,0.444702,0.987374,0.609566,0.279643,0.980609,0.817787,0.480741,0.279605
4,0.757521,0.502225,0.502225,0.38666,0.355246,0.771853,0.413771,0.671273
5,0.20075,0.947523,0.00398774,0.430645,0.599754,0.235945,0.145514,0.725926
6,0.127764,0.266831,0.906741,0.267865,0.0108293,0.637281,0.66997,0.655019
7,0.725646,0.782385,0.594094,0.33725,0.136704,0.352221,0.833425,0.128562
8,0.0748551,0.273572,0.238402,0.77553,0.108553,0.785024,0.316809,0.173973
9,0.578663,0.19875,0.191257,0.243014,0.39303,0.481783,0.37758,0.300757


In [43]:
ec.x1 # you can access columns of a parent data frame directly

10-element Array{Float64,1}:
 0.6995369083921843
 0.859555811172676
 0.44470235873549924
 0.7575213122413824
 0.20075005400832024
 0.12776437720077927
 0.7256461302224049
 0.07485514104662538
 0.5786633548942388
 0.08459176220407594